In [1]:
import sys
import pandas as pd
sys.path.append('/home/wangs/rs/lib')
# sys.path.append('/home/wangs/rs/qza')
import os
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import zipfile
import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import imp
import ff

In [2]:
# 所有加密货币的名字
currency_names=['BTC','ETH','XRP','BNB','TRX','ADA','ICP','APE','ARB','ARPA','ATOM','AVAX','BAKE','DOT','ETH',"FIL",'INJ','LINK','MKR','NEAR','NEO','RUNE','SEI','SOL','SUI','UNI','VET','WLD']
# currency_names = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'XLM', 'TRX', 'ADA']
# 读取一个币种的高开低收成交量
coin = "BTC"
ff.read_binance(coin)
# 预处理要用到的工具函数
freq = 15 # 15分钟为周期采样
resample_rule = "{}min".format(freq)

# Convert index from str into pd.Datetime
def timing(df):
    df.index = pd.to_datetime(df.index)
    df.index.name = None
    return df

def naming(df, name):
    df.name = name
    return df

def phase_transformer(s): # s = "20220101 010300"
    minute = int(s[11:13])
    return f"{s[:11]}{minute-minute%freq:02d}"

start, end = "20200101", "20240101"
# 读取数据
def currency_data(currency_name):
    global start, end
    data = ff.read_binance(currency_name).rename(columns={'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume'}).loc[start:end]
    data['tradedate'] = data.index.str[:8]
    data['tradehour'] = data.index.str[:11]
    data['tradephase'] = data.index.str[:15]
    data['tradephase'] = data.tradephase.apply(phase_transformer) # 如果要重采样的话，使用函数phase_transformer进行时间切分
    data = timing(data) # 将data的行索引转化为pd.datetime形式
    return data.reindex(pd.date_range(start=pd.to_datetime(start), 
                                      end=pd.to_datetime(end), 
                                      freq="1min", inclusive="left")) # 补充上缺失的时间索引

In [47]:
currency_names = list(set(currency_names))
data_all = [currency_data(currency_name)['volume'] for currency_name in currency_names]
data_all = pd.DataFrame(data_all).T
data_all.columns = currency_names

In [60]:
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
def viola(currency):
    data = data_all[currency].rolling(240).std()
    day_data = data[data.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
    return (day_data.rolling(20).std() / day_data.rolling(20).mean()).values
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(viola,currency_names),total = len(currency_names)))
vio_currency4 = pd.DataFrame(np.vstack(res_lst),columns=data_all.index.strftime("%Y-%m-%d %H:%M:%S")[data_all.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))],index=currency_names)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:05<00:00,  4.61it/s]


In [61]:
filter_index = ['00:00:00']
def viola(currency):
    data = data_all[currency].rolling(1440).std()
    day_data = data[data.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
    return (day_data.rolling(20).std() / day_data.rolling(20).mean()).values
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(viola,currency_names),total = len(currency_names)))
vio_currency24 = pd.DataFrame(np.vstack(res_lst),columns=data_all.index.strftime("%Y-%m-%d %H:%M:%S")[data_all.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))],index=currency_names)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:05<00:00,  4.80it/s]


In [80]:
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
def crest(currency):
    volume = data_all[currency]
    data = volume.rolling(240).mean() + volume.rolling(240).std()
    day_data = data[data.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
    judge_data = (pd.Series(volume.values - day_data.repeat(240).values,index = data.index))
    pre_data = judge_data.copy()
    pre_data[pre_data.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))] = 0
    crest = ((pre_data > 0).shift(1).replace({False:True,True:np.nan}) * (judge_data > 0).replace({False:np.nan}))
    crest_sum = crest.replace({np.nan:0}).rolling(240).sum()
    crest_sum = crest_sum[crest_sum.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
    crest_sum.index = pd.to_datetime(crest_sum.index).strftime('%Y%m%d')
    return crest_sum
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(crest,currency_names),total = len(currency_names)))
crest4 = pd.DataFrame(np.vstack(res_lst),columns=data_all.index.strftime("%Y-%m-%d %H:%M:%S")[data_all.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))],index=currency_names)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:19<00:00,  1.38it/s]


In [82]:
crest4

,2020-01-01 00:00:00,2020-01-01 04:00:00,2020-01-01 08:00:00,2020-01-01 12:00:00,2020-01-01 16:00:00,2020-01-01 20:00:00,2020-01-02 00:00:00,2020-01-02 04:00:00,2020-01-02 08:00:00,2020-01-02 12:00:00,...,2023-12-30 08:00:00,2023-12-30 12:00:00,2023-12-30 16:00:00,2023-12-30 20:00:00,2023-12-31 00:00:00,2023-12-31 04:00:00,2023-12-31 08:00:00,2023-12-31 12:00:00,2023-12-31 16:00:00,2023-12-31 20:00:00
INJ,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,15.0,9.0,36.0,16.0,13.0,32.0,9.0,24.0,9.0
ETH,NaN,0.0,2.0,26.0,5.0,29.0,15.0,19.0,15.0,12.0,...,4.0,1.0,10.0,43.0,14.0,17.0,9.0,7.0,16.0,48.0
ATOM,NaN,0.0,5.0,24.0,7.0,18.0,21.0,10.0,4.0,26.0,...,2.0,2.0,24.0,34.0,8.0,15.0,6.0,12.0,20.0,16.0
SOL,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,17.0,11.0,17.0,11.0,19.0,4.0,28.0,16.0,13.0
ARPA,NaN,0.0,6.0,44.0,26.0,8.0,12.0,17.0,20.0,12.0,...,9.0,1.0,15.0,1.0,23.0,4.0,13.0,14.0,6.0,13.0
NEO,NaN,1.0,2.0,7.0,9.0,12.0,4.0,11.0,3.0,19.0,...,7.0,13.0,28.0,11.0,5.0,7.0,4.0,30.0,3.0,14.0
DOT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,23.0,14.0,10.0,20.0,8.0,20.0,7.0,3.0
ADA,NaN,0.0,12.0,3.0,9.0,11.0,27.0,4.0,29.0,10.0,...,6.0,8.0,7.0,27.0,17.0,9.0,8.0,11.0,12.0,19.0
WLD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,34.0,12.0,26.0,36.0,5.0,7.0,13.0,4.0,26.0
NEAR,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,9.0,12.0,46.0,26.0,8.0,7.0,28.0,4.0,28.0
